In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Charger le dataset MNIST
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

# Redimensionner les images pour ajouter la dimension "canal" et normaliser
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Définir l'architecture du modèle CNN
model = models.Sequential([
    # Première couche convolutionnelle avec 32 filtres de taille 3x3
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Deuxième couche convolutionnelle avec 64 filtres
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Troisième couche convolutionnelle
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    
    # Aplatir les caractéristiques pour les passer à une couche dense
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes pour les chiffres de 0 à 9
])

# Compiler le modèle
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entraîner le modèle
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Évaluer le modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Précision sur le test :', test_acc)

# Sauvegarder le modèle en format HDF5
model.save('mon_modele.h5')



C:\Users\seben\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 12ms/step - accuracy: 0.8920 - loss: 0.3451 - val_accuracy: 0.9828 - val_loss: 0.0526
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9852 - loss: 0.0484 - val_accuracy: 0.9885 - val_loss: 0.0369
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9904 - loss: 0.0312 - val_accuracy: 0.9880 - val_loss: 0.0374
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9918 - loss: 0.0263 - val_accuracy: 0.9923 - val_loss: 0.0263
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9941 - loss: 0.0201 - val_accuracy: 0.9875 - val_loss: 0.0418
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9822 - loss: 0.0594


Précision sur le test : 0.987500011920929


In [15]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageDraw
import numpy as np
import tensorflow as tf

class DrawingApp(tk.Tk):
    def __init__(self, model):
        super().__init__()
        self.title("Reconnaissance de Chiffres")
        self.model = model
        self.canvas_width = 200
        self.canvas_height = 200

        # Création du canevas pour dessiner
        self.canvas = tk.Canvas(self, width=self.canvas_width, height=self.canvas_height, bg='white')
        self.canvas.pack(padx=10, pady=10)

        # Bouton pour prédire le chiffre
        self.btn_predict = tk.Button(self, text="Prédire", command=self.predict_digit)
        self.btn_predict.pack(side='left', padx=10)

        # Bouton pour effacer le dessin
        self.btn_clear = tk.Button(self, text="Effacer", command=self.clear_canvas)
        self.btn_clear.pack(side='right', padx=10)

        # Liaison de l'événement de dessin
        self.canvas.bind("<B1-Motion>", self.draw)
        self.canvas.bind("<ButtonRelease-1>", self.reset)

        # Création d'une image PIL pour pouvoir la traiter ensuite
        self.image1 = Image.new("L", (self.canvas_width, self.canvas_height), color=255)
        self.draw1 = ImageDraw.Draw(self.image1)
        self.last_x, self.last_y = None, None

    def draw(self, event):
        x, y = event.x, event.y
        if self.last_x is not None and self.last_y is not None:
            # Dessiner sur le canevas Tkinter
            self.canvas.create_line(self.last_x, self.last_y, x, y, fill="black", width=8)
            # Dessiner sur l'image PIL
            self.draw1.line([self.last_x, self.last_y, x, y], fill=0, width=8)
        self.last_x, self.last_y = x, y

    def reset(self, event):
        self.last_x, self.last_y = None, None

    def clear_canvas(self):
        self.canvas.delete("all")
        self.image1 = Image.new("L", (self.canvas_width, self.canvas_height), color=255)
        self.draw1 = ImageDraw.Draw(self.image1)

    def predict_digit(self):
        # Prétraitement de l'image :
        # Redimensionner l'image en 28x28 pixels
        img = self.image1.resize((28, 28))
        # Convertir en tableau numpy
        img_array = np.array(img)
        # Inverser les couleurs pour correspondre aux images MNIST (fond noir, chiffre blanc)
        img_array = 255 - img_array
        # Normaliser les valeurs entre 0 et 1
        img_array = img_array / 255.0
        # Reshape pour le modèle (1, 28, 28)
        img_array = img_array.reshape(1, 28, 28)

        # Faire la prédiction avec le modèle
        prediction = self.model.predict(img_array)
        predicted_digit = np.argmax(prediction)

        # Afficher le résultat
        messagebox.showinfo("Résultat", f"Le chiffre prédit est : {predicted_digit}")

def main():
    # Charger le modèle sauvegardé (par exemple 'mon_modele.h5')
    model = tf.keras.models.load_model("mon_modele.h5")
    app = DrawingApp(model)
    app.mainloop()

if __name__ == "__main__":
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
